In [158]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

import torch
# Assuming you have your data in X (features) and y (target)
# X should be a concatenation of title embedding and description embedding
# y should contain the target labels, where each label is a string containing comma-separated tags


In [134]:
dataset = pd.read_csv('embedded_dataset.csv')

In [135]:
tags = pd.read_csv('data/llm_tags_mini.csv').iloc[:20]

In [136]:
dataset_mini = dataset.iloc[:20]

In [137]:
concatenated_dataset = pd.concat([dataset_mini, tags['llm_tags']], axis=1)

In [138]:
indexes = []
for i in range(20):
    if concatenated_dataset['llm_tags'].iloc[i] == '[]':
        indexes.append(i)
print(indexes)

[11, 15]


In [139]:
title_embeddings = torch.load('title_embeddings.pt')


In [140]:
for i in range(len(title_embeddings)):
    title_embeddings[i] = title_embeddings[i].numpy()

In [145]:
concatenated_dataset.head(1)

,URL,Title,Description,Article Text,Keywords,Tag,Description_embeddings,Title_embeddings,llm_tags,Title_tensors,description_tensors
0,https://kino.mail.ru/news/61905-serial-na-viho...,Сериал на выходные: едкая детективная комедия ...,В элитном поселке происходит убийство няни в ...,"В элитном поселке, где обитают сплошь богачи, ...","СТАТЬЯ, ЧТО ПОСМОТРЕТЬ, ДРАМА, КОМЕДИЯ, ДЕТЕКТИВ",Кино,"tensor([[ 1.9462e-01, -4.3563e-02, 3.7849e-01...","tensor([[ 6.6988e-02, -1.4729e-01, 2.5354e-01...","['Кино', 'Телевидение', 'Персоны']","[[0.06698765, -0.14728764, 0.2535393, 0.545502...","[[0.19461995, -0.043562844, 0.37849262, 0.0168..."


In [142]:
title_embeddings = title_embeddings[:20]

# Convert the title embeddings to a pandas Series
title_embeddings_series = pd.Series(title_embeddings, name='Title_tensors')

# Concatenate the title embeddings to the dataset
concatenated_dataset = pd.concat([concatenated_dataset, title_embeddings_series], axis=1)

# Rename the column if needed
# concatenated_dataset.rename(columns={0: 'title_tensors'}, inplace=True)

In [144]:
description_embeddings = torch.load('description_embeddings.pt')
for i in range(len(description_embeddings)):
    description_embeddings[i] = description_embeddings[i].numpy()
description_embeddings = description_embeddings[:20]

# Convert the title embeddings to a pandas Series
description_embeddings_series = pd.Series(description_embeddings, name='description_tensors')

# Concatenate the title embeddings to the dataset
concatenated_dataset = pd.concat([concatenated_dataset, description_embeddings_series], axis=1)


In [146]:
concatenated_dataset.columns

Index(['URL', 'Title', 'Description', 'Article Text', 'Keywords', 'Tag',
       'Description_embeddings', 'Title_embeddings', 'llm_tags',
       'Title_tensors', 'description_tensors'],
      dtype='object')

In [147]:
concatenated_dataset.drop(indexes, axis=0, inplace=True)

In [148]:
for col in concatenated_dataset.columns:
    if col not in ['description_tensors', 'Title_tensors', 'llm_tags']:
        concatenated_dataset.drop(col, axis=1, inplace=True)

In [46]:
# import ast
# concatenated_dataset['llm_tags'] = concatenated_dataset['llm_tags'].apply(ast.literal_eval)

In [150]:
X = concatenated_dataset.drop('llm_tags', axis=1)
y = concatenated_dataset['llm_tags']

In [168]:
for i in range(len(X.Title_tensors)):
    X.Title_tensors.iloc[i] = X.Title_tensors.iloc[i].flatten()

for i in range(len(X.description_tensors)):
    X.description_tensors.iloc[i] = X.description_tensors.iloc[i].flatten()

In [171]:
y

0     ['Кино', 'Телевидение', 'Персоны']
1        ['Кино', 'События', 'Общество']
2         ['Кино', 'Персоны', 'События']
3     ['Кино', 'Телевидение', 'Персоны']
4     ['Кино', 'Телевидение', 'Персоны']
5         ['Кино', 'Персоны', 'События']
6         ['Кино', 'События', 'Персоны']
7         ['Кино', 'Персоны', 'События']
8         ['Кино', 'Персоны', 'События']
9         ['Кино', 'Персоны', 'События']
10        ['Кино', 'События', 'Персоны']
12        ['Кино', 'Персоны', 'События']
13    ['Кино', 'Телевидение', 'Персоны']
14    ['Кино', 'Телевидение', 'Персоны']
16        ['Кино', 'Персоны', 'События']
17        ['Кино', 'Персоны', 'События']
18    ['Кино', 'Телевидение', 'Персоны']
19        ['Кино', 'Персоны', 'События']
Name: llm_tags, dtype: object

In [176]:
y = y.apply(lambda x: x.split(', ')) 

In [177]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Assuming X and y are your feature and target arrays

# Transforming the target labels into a binary format suitable for multi-label classification
mlb = MultiLabelBinarizer(classes=['Кино', 'Телевидение', 'Персоны', 'События', 'Общество'])
y_binarized = mlb.fit_transform(y)  # Fit and transform on the entire target labels

# Splitting the data into training and testing sets
X_train, X_test, y_train_binarized, y_test_binarized = train_test_split(X, y_binarized, test_size=0.2, random_state=42)

# Training the Multinomial Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train_binarized)

# Making predictions on the test set
predictions = naive_bayes_classifier.predict(X_test)

# Evaluating the accuracy
accuracy = accuracy_score(y_test_binarized, predictions)
print("Accuracy:", accuracy)


/Users/riter/anaconda3/envs/nlp/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ["'Общество']", "'Персоны'", "'Персоны']", "'События'", "'События']", "'Телевидение'", "['Кино'"] will be ignored
  warnings.warn(


ValueError: setting an array element with a sequence.